In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import torch
print(torch.__version__)
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
import random
import cv2
import numpy as np
!pip install pyefd
import pyefd
from google.colab.patches import cv2_imshow
!pip install cairocffi
import cairocffi as cairo
import struct
from struct import unpack

1.12.1+cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 4.5 MB/s 
  Created wheel for cairocffi: filename=cairocffi-1.3.0-py3-none-any.whl size=89668 sha256=0b819526ed050ff682874c62c34fbc7bf98f92e0d63086a10681803abc435301
  Stored in directory: /root/.cache/pip/wheels/4e/ca/e1/5c8a9692a27f639a07c949044bec943f26c81cd53d3805319f
Successfully built cairocffi


Section 1: Fourier Descriptors through linear classifier

In [8]:
# Env vars
torch.use_deterministic_algorithms(False)

# Const vars
LOAD_PATH = '/content/drive/My Drive/Fourier/Saved Models/MLP vector single noconn.pt'
RAND_SEED = 0
DEVICE = "cpu"

IMG_SIDE = 28
NUM_CLASSES = 343
EPOCHS = 20
BATCH_SIZE = 512
LOSS_FN = nn.CrossEntropyLoss()

In [4]:
def get_step(deltaX, deltaY):
  angle = np.arctan2(deltaY, deltaX)
  pi_over_8 = np.pi / 8
  if -pi_over_8 <= angle < pi_over_8:
    return (1, 0)
  elif pi_over_8 <= angle < 3 * pi_over_8:
    return (1, 1)
  elif 3 * pi_over_8 <= angle < 5 * pi_over_8:
    return (0, 1)
  elif 5 * pi_over_8 <= angle < 7 * pi_over_8:
    return (-1, 1)
  elif angle >= 7 * pi_over_8 or angle < -7 * pi_over_8:
    return (-1, 0)
  elif -7 * pi_over_8 <= angle < -5 * pi_over_8:
    return (-1, -1)
  elif -5 * pi_over_8 <= angle < -3 * pi_over_8:
    return (0, -1)
  else:
    return (1, -1)

def complete_contour(vector_img):
  complete_contour = []
  partial_contour = [point for stroke in vector_img for point in zip(*stroke)]
  if len(partial_contour) > 0:
    partial_contour.append(partial_contour[0])
    for i in range(len(partial_contour) - 1):
      x0, y0 = partial_contour[i]
      x1, y1 = partial_contour[i+1]
      if (x0, y0) == (x1, y1):
        continue
      complete_contour.append([x0, y0])
  return np.asarray(complete_contour)

means = np.asarray([[1.00000000e+00,  1.70292625e-19,  1.37157261e-19, -2.97352773e-02],
 [ 3.93857845e-02,  1.14573042e-02,  1.11273905e-02, -1.27501456e-02],
 [ 1.26057258e-02,  1.27047894e-02,  2.20446418e-03, -5.17296120e-03],
 [ 4.60867346e-03,  2.54431263e-04, -2.24875991e-03, -2.43568670e-03],
 [-1.29011658e-03,  7.29537187e-04,  3.29306713e-04, -6.94848183e-05],
 [-2.20461684e-03, -4.26032423e-04,  3.70747798e-04,  6.67042619e-04],
 [ 1.06105248e-03,  5.59965312e-04, -2.48033261e-05, -3.27838867e-04],
 [ 6.92991267e-04,  7.77918380e-05,  1.26802111e-04, -4.82832044e-04],
 [ 2.05231401e-04, -1.29655496e-04, -1.44908870e-04, -2.86502180e-04],
 [-1.48524853e-04, -2.01465364e-04,  2.28534063e-04,  3.38029409e-04]])

stdevs = np.asarray([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 3.94827207e-01],
 [6.15715499e-01, 6.31045641e-01, 5.56262445e-01, 5.77054655e-01],
 [4.44723755e-01, 4.34617210e-01, 4.14754373e-01, 4.09847350e-01],
 [3.13965726e-01, 3.06039399e-01, 2.93664393e-01, 2.88944168e-01],
 [2.20598291e-01, 2.20428499e-01, 2.08627310e-01, 2.08939754e-01],
 [1.67779200e-01, 1.70621159e-01, 1.61133731e-01, 1.60840793e-01],
 [1.35504574e-01, 1.33753600e-01, 1.28539543e-01, 1.29087092e-01],
 [1.08136940e-01, 1.08601151e-01, 1.04438585e-01, 1.03887367e-01],
 [9.21341521e-02, 9.22050837e-02, 8.90123677e-02, 8.74814102e-02],
 [7.74686779e-02, 7.69117614e-02, 7.27067303e-02, 7.27266299e-02]])

# transform functions - take sketch image, return torch tensor of descriptors
def fourier_transform(vector_img, is_test):
  contour = complete_contour(vector_img)

  # to-do
  # if is_test:
  
  coeffs = pyefd.elliptic_fourier_descriptors(contour, order=FOURIER_ORDER, normalize=True)
  coeffs = (coeffs - means) / stdevs
  return torch.from_numpy(coeffs.flatten()).float()

# helper method to find class based on imgset index
def find_class(idx, num_list):
  class_id = 0
  sum = num_list[class_id]
  while idx >= sum:
    class_id += 1
    sum += num_list[class_id]
  return class_id

# deterministic worker re-seeding
def seed_worker(worker_id):
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

# custom dataset for quickdraw
class QuickdrawDataset(Dataset):
  def __init__(self, imgs, nums, is_test):
    self.imgs = imgs
    self.nums = nums
    self.len = sum(nums)
    self.is_test = is_test

  def __len__(self):
    return self.len

  def __getitem__(self, idx):
    img = self.imgs[idx]
    x = fourier_transform(img, self.is_test)
    y = find_class(idx, self.nums)
    return x, y


class MLP(torch.nn.Module):
  def __init__(self):
    super(MLP, self).__init__()
    self.fc1 = nn.Linear(FOURIER_ORDER * 4, 464)
    self.fc2 = nn.Linear(464, 464)
    self.fc3 = nn.Linear(464, 464)
    self.head = nn.Linear(464, NUM_CLASSES)
    self.relu = nn.ReLU()


  def forward(self, x):
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    x = self.relu(x)
    x = self.fc3(x)
    x = self.relu(x)
    return self.head(x)


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # put the model in train mode
    total_loss = 0
    total_correct = 0
    # for each batch in the training set compute loss and update model parameters
    for batch, (x, y) in enumerate(dataloader):
      x, y = x.to(DEVICE), y.to(DEVICE)
      # Compute prediction and loss
      out = model(x)
      loss = loss_fn(out, y)

      # Backpropagation to update model parameters
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # print current training metrics for user
      y, out, loss = y.to("cpu"), out.to("cpu"), loss.to("cpu")
      loss_val = loss.item()
      if batch % 100 == 0:
          current = (batch + 1) * BATCH_SIZE
          print(f"loss: {loss_val:>7f}  [{current:>5d}/{size:>5d}]")

      pred = out.argmax(dim=1, keepdim=True)
      correct = pred.eq(y.view_as(pred)).sum().item()
      total_correct += correct
      total_loss += loss_val
      # print(f"train loss: {loss_val:>7f}   train accuracy: {correct / BATCH_SIZE:>7f}   [batch: {batch + 1:>3d}/{(size // BATCH_SIZE) + 1:>3d}]")      
    print(f"\nepoch avg train loss: {total_loss / ((size // BATCH_SIZE) + 1):>7f}   epoch avg train accuracy: {total_correct / size:>7f}")
      
def eval_loop(dataloader, model):
  model.eval()
  size = len(dataloader.dataset)
  with torch.no_grad():
    total_correct = 0
    for x, y in dataloader:
      x, y = x.to(DEVICE), y.to(DEVICE)
      out = model(x)
      y, out = y.to("cpu"), out.to("cpu")
      pred = out.argmax(dim=1, keepdim=True)
      total_correct += pred.eq(y.view_as(pred)).sum().item()

    accuracy = total_correct / size
    print(f"test accuracy: {accuracy:>7f}")


In [5]:
# define methods for unpacking Quickdraw .bin files
def unpack_drawing(file_handle):
  file_handle.read(15)
  n_strokes, = unpack('H', file_handle.read(2))
  image = []
  for i in range(n_strokes):
      n_points, = unpack('H', file_handle.read(2))
      fmt = str(n_points) + 'B'
      x = unpack(fmt, file_handle.read(n_points))
      y = unpack(fmt, file_handle.read(n_points))
      image.append((x, y))

  return image


def unpack_drawings(filename):
  imageset = []
  with open(filename, 'rb') as f:
      while True:
          try:
              imageset.append(unpack_drawing(f))
          except struct.error:
              break
  return imageset

train_dir = '/content/drive/My Drive/Fourier/Quickdraw Dataset Small/Train/'
test_dir = '/content/drive/My Drive/Fourier/Quickdraw Dataset Small/Test/'
train_imgs = []
test_imgs = []
train_nums = []
test_nums = []
list_of_classes = ["The Eiffel Tower", "The Great Wall of China", "The Mona Lisa",
                   "aircraft carrier", "airplane", "alarm clock", "ambulance", 
                   "angel", "ant", "anvil", "apple", "arm", "asparagus", "axe", 
                   "backpack", "banana", "bandage", "barn", "baseball bat", 
                   "baseball", "basket", "basketball", "bathtub", "beach", "bear", 
                   "beard", "bed", "bee", "belt", "bench", "bicycle", "binoculars", 
                   "bird", "birthday cake", "blackberry", "blueberry", "book", 
                   "boomerang", "bottlecap", "bowtie", "bracelet", "brain", 
                   "bread", "bridge", "broccoli", "broom", "bucket", "bulldozer", 
                   "bus", "bush", "butterfly", "cactus", "cake", "calculator", 
                   "calendar", "camel", "camera", "camouflage", "campfire", 
                   "candle", "cannon", "canoe", 'car', 'carrot', "castle", "cat", "ceiling fan", 
                   "cell phone", "cello", "chair", "chandelier", "church", 
                   "circle", "clarinet", "clock", "cloud", "coffee cup", 
                   "compass", "computer", "cookie", "cooler", "couch", "cow",
                   "crab", "crayon", "crocodile", "crown", "cruise ship", 
                   "cup", "diamond", "dishwasher", "diving board", "dog", 
                   "dolphin", "donut", "door", "dragon", "dresser", "drill", 
                   "drums", "duck", "dumbbell", "ear", "elbow", "elephant", 
                   "envelope", "eraser", "eye", "eyeglasses", "face", "fan",
                   "feather", "fence", "finger", "fire hydrant", "fireplace",
                   "firetruck", "fish", "flamingo", "flashlight", "flip flops", 
                   "floor lamp", "flower", "flying saucer", "foot", "fork", 
                   "frog", "frying pan", "garden hose", "garden", "giraffe", 
                   "goatee", "golf club", "grapes", "grass", "guitar", 
                   "hamburger", "hammer", "hand", "harp", "hat", "headphones", 
                   "hedgehog", "helicopter", "helmet", "hexagon", "hockey puck", 
                   "hockey stick", "horse", "hospital", "hot air balloon", 
                   "hot dog", "hot tub", "hourglass", "house plant", "house", 
                   "hurricane", "ice cream", "jacket", "jail", "kangaroo", 
                   "key", "keyboard", "knee", "knife", "ladder", "lantern", 
                   "laptop", "leaf", "leg", "light bulb", "lighter", "lighthouse",
                   "lightning", "line", "lion", "lipstick", "lobster", "lollipop",
                   "mailbox", "map", "marker", "matches", "megaphone", "mermaid", 
                   "microphone", "microwave", "monkey", "moon", "mosquito", 
                   "motorbike", "mountain", "mouse", "moustache", "mouth", "mug",
                   "mushroom", "nail", "necklace", "nose", "ocean", "octagon", 
                   "octopus", "onion", "oven", "owl", "paint can", "paintbrush", 
                   "palm tree", "panda", "pants", "paper clip", "parachute", 
                   "parrot", "passport", "peanut", "pear", "peas", "pencil", 
                   "penguin", "piano", "pickup truck", "picture frame", "pig", 
                   "pillow", "pineapple", "pizza", "pliers", "police car", 
                   "pond", "pool", "popsicle", "postcard", "potato", 
                   "power outlet", "purse", "rabbit", "raccoon", "radio", 
                   "rain", 'rainbow', 'rake', 'remote control', 'rhinoceros', 
                   'rifle', 'river', 'roller coaster', 'rollerskates', 
                   'sailboat', 'sandwich', 'saw', 'saxophone', 'school bus', 
                   'scissors', 'scorpion', 'screwdriver', 'sea turtle', 
                   'see saw', 'shark', 'sheep', 'shoe', 'shorts', 'shovel', 
                   'sink', 'skateboard', 'skull', 'skyscraper', 'sleeping bag', 
                   'smiley face', 'snail', 'snake', 'snorkel', 'snowflake', 
                   'snowman', 'soccer ball', 'sock', 'speedboat', 'spider', 
                   'spoon', 'spreadsheet', 'square', 'squiggle', 'squirrel', 
                   'stairs', 'star', 'steak', 'stereo', 'stethoscope', 'stitches', 
                   'stop sign', 'stove', 'strawberry', 'streetlight', 
                   'string bean', 'submarine', 'suitcase', 'sun', 'swan', 
                   'sweater', 'swing set', 'sword', 'syringe', 't-shirt', 
                   'table', 'teapot', 'teddy-bear', 'telephone', 'television', 
                   'tennis racquet', 'tent', 'tiger', 'toaster', 'toe', 'toilet', 
                   'tooth', 'toothbrush', 'toothpaste', 'tornado', 'tractor', 
                   'traffic light', 'train', 'tree', 'triangle', 'trombone', 
                   'truck', 'trumpet', 'umbrella', 'underwear', 'van', 'vase', 'violin', 
                   'washing machine', 'watermelon', 'waterslide', 'whale', 
                   'wheel', 'windmill', 'wine bottle', 'wine glass', 'wristwatch', 
                   'yoga', 'zebra', 'zigzag']

In [6]:
# load dataset
for item in list_of_classes:
  train_folder = train_dir + item + '.bin'
  # test_folder = test_dir + item + '.bin'
  train_drawings = unpack_drawings(train_folder)
  train_imgs += train_drawings
  train_nums.append(len(train_drawings))
  # test_drawings = unpack_drawings(test_folder)
  # test_imgs += test_drawings
  # test_nums.append(len(test_drawings))

In [9]:
FOURIER_ORDER = 10

# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_fourier_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_fourier_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
# test_fourier_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_fourier_loader = DataLoader(train_fourier_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_fourier_loader = DataLoader(eval_fourier_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
# test_fourier_loader = DataLoader(test_fourier_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = MLP()
# checkpoint = torch.load(LOAD_PATH, map_location=torch.device(DEVICE))
# model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
lr = 0.001
optim = torch.optim.Adam(model.parameters(), lr=lr)
# optim.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
epoch = 0
print("\n\n\nFourier order is: "+str(FOURIER_ORDER)+"\n\n\n")

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
# eval_loop(dataloader=test_fourier_loader,model=model)
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/MLP vector single noconnF10-001.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_fourier_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_fourier_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
# random.seed(RAND_SEED)
# eval_loop(dataloader=test_fourier_loader,model=model)




Fourier order is: 10



Epoch 1

loss: 5.837706  [  512/342985]
loss: 5.490666  [51712/342985]
loss: 5.439081  [102912/342985]
loss: 5.179988  [154112/342985]
loss: 5.077914  [205312/342985]
loss: 4.976070  [256512/342985]
loss: 4.842170  [307712/342985]

epoch avg train loss: 5.182772   epoch avg train accuracy: 0.049145

-------------------------------

Epoch 2

loss: 4.728156  [  512/342985]
loss: 4.696208  [51712/342985]
loss: 4.581374  [102912/342985]
loss: 4.582870  [154112/342985]
loss: 4.469681  [205312/342985]
loss: 4.629896  [256512/342985]
loss: 4.632366  [307712/342985]

epoch avg train loss: 4.548032   epoch avg train accuracy: 0.123090

-------------------------------

Epoch 3

loss: 4.461313  [  512/342985]
loss: 4.299780  [51712/342985]
loss: 4.203264  [102912/342985]
loss: 4.205159  [154112/342985]
loss: 4.286357  [205312/342985]
loss: 4.245231  [256512/342985]
loss: 4.261765  [307712/342985]

epoch avg train loss: 4.252590   epoch avg train accuracy: 0.163409

----